## PyTorch Image Prediction (with Convolutional Neural Network)

### Implementing Tensor

#### Import Libraries

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

In [ ]:
training_data = datasets.MNIST(root=".", train=True, download=True, transform=ToTensor())
# training_data = datasets.FashionMNIST(root=".", train=True, download=True, transform=ToTensor())

test_data = datasets.MNIST(root=".", train=False, download=True, transform=ToTensor())
# test_data = datasets.FashionMNIST(root=".", train=False, download=True, transform=ToTensor())

In [ ]:
print(torchvision.__version__)

In [ ]:
training_data.classes

In [ ]:
from torch.utils.data import DataLoader

batch_size = 32

loaded_train = DataLoader(training_data, batch_size=batch_size, shuffle=True)
loaded_test = DataLoader(test_data, batch_size=batch_size, shuffle=True)

### Creating Neural Network Model

In [ ]:
class MNISTModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Convolutional Layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)  # Input 1x28x28, Output 32x28x28
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output 32x14x14

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  # Input 32x14x14, Output 64x14x14
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output 64x7x7

        # Fully Connected Layers
        self.fc1 = nn.Linear(64 * 7 * 7, 512)  # Input 64*7*7=3136, Output 512
        self.act3 = nn.ReLU()
        self.fc2 = nn.Linear(512, 10)  # Input 512, Output 10 (number of classes)

    def forward(self, x):
        x = self.pool1(self.act1(self.conv1(x)))
        x = self.pool2(self.act2(self.conv2(x)))
        x = torch.flatten(x, 1)  # Flatten all dimensions except batch
        x = self.act3(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
model = MNISTModel()
print(model)

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

### Train & Test Model

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    for batch_idx, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            current = batch_idx * len(X)
            print(f"Loss: {loss.item():.6f}  [{current}/{size}]")

    print("Training complete!")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Accuracy: {(100*correct):>0.1f}")
epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(loaded_train, model, loss_function, optimizer)
    test(loaded_test, model, loss_function)
print("Done!")

### Save Model

In [ ]:
torch.save(model.state_dict(), 'cpu_trained_model.pth')

# PyTorch in Web App with Flask

In [ ]:
import torch

print(torch.__version__)

### Import Python's Flask Library

In [ ]:
from flask import Flask, render_template, request
from PIL import Image
from io import BytesIO
import torch
from torchvision.transforms import ToTensor

### Model pre-trained in Local/VSCode (epochs=1, accuracy>60%)

In [ ]:
# Load your trained model
model = MNISTModel()

print(model)

### Model pre-trained in Google Colab (epochs=5, accuracy>90%)

In [ ]:
# Load Trained Model

device = torch.device('cpu')
model = MNISTModel()

PATH = 'trained_model_acc_90.pth'
model.load_state_dict(torch.load(PATH, map_location=device))

### Flask Back-End

In [ ]:
from flask import Flask, render_template, request
from PIL import Image
from torchvision.transforms import ToTensor
from io import BytesIO
import torch

app = Flask(__name__, template_folder='templates', static_url_path='/static')

def initialize_flask_app():
    # Define route (endpoint) for home page
    @app.route('/', methods=['GET'])
    def home():
        return render_template('index.html')

    # Define route (endpoint) for handling image upload and prediction
    @app.route('/predict', methods=['POST'])
    def predict():
        if 'image' in request.files:
            # Read and preprocess the uploaded image
            img = Image.open(BytesIO(request.files['image'].read()))
            img = img.convert('L')  # Convert to grayscale
            img = img.resize((28, 28))  # Resize image to 28x28
            img = ToTensor()(img).unsqueeze(0)  # Convert to tensor and add batch dimension

            # Perform prediction using your model
            with torch.no_grad():
                output = model(img)
                _, predicted = torch.max(output, 1)
                prediction = predicted.item()

            return render_template('result.html', prediction=prediction)
        else:
            return 'Error: No image provided.'

    # Run the Flask app
    if __name__ == '__main__':
        app.run()

if __name__ == '__main__':
    initialize_flask_app()
